In [ ]:
import os
import random
import re

import numpy as np
import pandas as pd
import spacy
import torch
import transformers
import wandb
from datasets import Dataset, load_dataset
from pandarallel import pandarallel
from sklearn import metrics
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import MultiLabelBinarizer
from torch import cuda
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
from tqdm.notebook import tqdm
from transformers import (
    AutoTokenizer,
    BertConfig,
    BertForSequenceClassification,
    BertModel,
    BertTokenizer,
    DataCollatorWithPadding,
    T5ForConditionalGeneration,
    T5Tokenizer,
)

np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

pandarallel.initialize(progress_bar=True)

device = "cuda" if cuda.is_available() else "cpu"


nlp = spacy.load("ru_core_news_lg")

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/seara/Desktop/Github/mtc-film-classification/.venv/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'ru_core_news_lg' (3.7.0) was trained with spaCy v3.7.0 and may not be 100% compatible with the current version (3.8.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
df = pd.read_csv("data/train.csv")
df

,Фильм,Описание,Сюжет,Жанры
0,Дивергент (2014),"Действие фильма «Дивергент» происходит в мире,...","Действие фильма «Дивергент» происходит в мире,...","фантастика, детектив, боевик, мелодрама"
1,Кунг-фу Панда 4 (2024),Однажды ночью на вершине горы возле каменоломн...,Однажды ночью на вершине горы возле каменоломн...,"мультфильм, фэнтези, боевик, комедия, приключения"
2,2046 (2004),Чоу возвращается в Гонконг после нескольких ле...,Чоу возвращается в Гонконг после нескольких ле...,"фантастика, драма, мелодрама"
3,Полицейский из Беверли-Хиллз: Аксель Фоули (2024),Аксель Фоули вернулся в Беверли-Хиллз после то...,Аксель Фоули вернулся в Беверли-Хиллз после то...,"боевик, комедия, криминал, детектив"
4,"Знакомьтесь, Джо Блэк (1998)","История об Ангеле Смерти, который решает взять...","История об Ангеле Смерти, который решает взять...","мелодрама, фэнтези, драма"
...,...,...,...,...
566,Апокалипсис (2006),В 1517 году на полуострове Юкатан племя Лапы Я...,1517 год. Полуостров Юкатан. Группа охотников ...,"боевик, триллер, драма, приключения"
567,Лёд 3 (2024),"Надя, ставшая фигуристкой, стремится выиграть ...",Фильм начинается с истории взросления дочери А...,"мюзикл, мелодрама"
568,Дастур (2023),"Новоиспеченная невеста, которую выдали замуж п...",NaN,"ужасы, фантастика"
569,Не говори никому (2024),Пара вместе с дочерью получают приглашение от ...,Пара вместе с дочерью получают приглашение от ...,"триллер, драма"


In [ ]:
df = df.rename(
    {"Фильм": "movie", "Сюжет": "plot", "Жанры": "genres", "Описание": "description"},
    axis=1,
)
# df["plot"] = df["plot"].fillna(df["description"])
df.isnull().any()

movie          False
description    False
plot            True
genres         False
dtype: bool

In [ ]:
df["genres"] = df["genres"].apply(lambda x: x.split(", "))
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df["genres"])
df["target"] = y.tolist()
df = df.drop("genres", axis=1)


def clear_text(text):
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("\xa0", " ")
    return " ".join(text.split())


df["plot"] = df["plot"].apply(lambda x: clear_text(x) if pd.notna(x) else x)
df["description"] = df["description"].apply(clear_text)

In [ ]:
df["plot_desc"] = df.apply(
    lambda row: (
        row["plot"] + " " + row["description"]
        if pd.notna(row["plot"])
        else row["description"]
    ),
    axis=1,
)

In [ ]:
MODEL_NAME = "cointegrated/rut5-base-absum"
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda()
model.eval()


def summarize(
    text,
    n_words=None,
    compression=None,
    max_length=10000,
    num_beams=3,
    do_sample=False,
    repetition_penalty=10.0,
    **kwargs,
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = f"[{n_words}] " + text
    elif compression:
        text = f"[{compression:.1g}] " + text
    x = tokenizer(text, return_tensors="pt", padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x,
            max_length=max_length,
            num_beams=num_beams,
            do_sample=do_sample,
            repetition_penalty=repetition_penalty,
            **kwargs,
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

/home/seara/Desktop/Github/mtc-film-classification/.venv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
df["plot_desc"][5]

'У Джесса Ааронса, замкнутого в себе мальчика, совсем нет друзей. В школе его постоянно задирают, семья, в которой, кроме него, есть две старшие и две младшие сестры, стеснена в средствах, а с отцом у него очень трудные отношения. Желая хоть в чём-то стать первым, он принимает участие в школьных состязаниях по бегу, но его обгоняет девочка по имени Лесли, которая только что перевелась в их класс. Её семья переехала в пригород, где живут Ааронсы, и поселилась с ними по соседству. Лесли, как и Джесс, тоже становится белой вороной из-за своего независимого характера. Вскоре они становятся лучшими друзьями и раскрывают друг другу свои секреты: Джесс показывает Лесли свои рисунки, а она рассказывает ему свои выдумки. Во время прогулки по лесу они перебираются через протекающую там речку на тарзанке и обнаруживают на другой стороне заброшенный домик на дереве. Этот домик становится их секретным местом, о котором знают лишь они двое. Здесь они создают целый сказочный мир и называют его Тераби

In [ ]:
summarize(df["plot_desc"][5], compression=0.1)

'В школе у Джесса Ааронса совсем нет друзей. Они объединяются против компании старших учениц во главе с Дженис Эйвери, которые вымогают у более слабых по сравнению с ними младшеклассников.'

'Действие фильма «Дивергент» происходит в мире, пережившем глобальную войну, в футуристическом Чикаго, где общество разделено на пять фракций: «Эрудиция» (Erudite), «Дружелюбие» (Amity), «Искренность» (Candor), «Отречение» (Abnegation) и «Бесстрашие» (Dauntless). Каждая из них объединяет в себе представителей определённого типа. Каждый взрослый житель, достигший 16 лет, должен пройти тест, чтобы определить, к какой фракции он принадлежит, а на посвящении либо довериться тесту и остаться в своей фракции, либо по зову души выбрать другую фракцию. Главная героиня Беатрис Прайор, проходя тест, обнаружила, что ей подходят сразу все фракции. Таких людей называют дивергентами. Общество очень боялось дивергентов, ибо они не умеют подчиняться и нестандартно мыслят. Не признаваясь, Беатрис выбирает «Бесстрашие», потому что всегда восхищалась этой фракцией. В результате она приобретает новый дом, новых друзей — Кристину, Уилла и Ала — и нового врага Питера, влюбляется в своего инструктора Фора и 

In [ ]:
df["plot_summary"] = df["plot"].apply(lambda x: summarize(x, n_words=100))

,movie,description,plot,target
0,Дивергент (2014),"Действие фильма «Дивергент» происходит в мире,...","Действие фильма «Дивергент» происходит в мире,...","[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]"
1,Кунг-фу Панда 4 (2024),Однажды ночью на вершине горы возле каменоломн...,Однажды ночью на вершине горы возле каменоломн...,"[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1]"
2,2046 (2004),Чоу возвращается в Гонконг после нескольких ле...,Чоу возвращается в Гонконг после нескольких ле...,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]"
3,Полицейский из Беверли-Хиллз: Аксель Фоули (2024),Аксель Фоули вернулся в Беверли-Хиллз после то...,Аксель Фоули вернулся в Беверли-Хиллз после то...,"[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"Знакомьтесь, Джо Блэк (1998)","История об Ангеле Смерти, который решает взять...","История об Ангеле Смерти, который решает взять...","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]"
...,...,...,...,...
566,Апокалипсис (2006),В 1517 году на полуострове Юкатан племя Лапы Я...,1517 год. Полуостров Юкатан. Группа охотников ...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]"
567,Лёд 3 (2024),"Надя, ставшая фигуристкой, стремится выиграть ...",Фильм начинается с истории взросления дочери А...,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
568,Дастур (2023),"Новоиспеченная невеста, которую выдали замуж п...","Новоиспеченная невеста, которую выдали замуж п...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]"
569,Не говори никому (2024),Пара вместе с дочерью получают приглашение от ...,Пара вместе с дочерью получают приглашение от ...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"


In [ ]:
def replace_entities(text):
    doc = nlp(text)
    modified_text = text
    for ent in doc.ents:
        if ent.label_ == "PER":
            modified_text = modified_text.replace(ent.text, "герой")
        elif ent.label_ == "LOC":
            modified_text = modified_text.replace(ent.text, "локация")
    return modified_text


text = "Аксёль Фоули вернулся в фильм Полицейский из Беверли-Хиллз"
processed_text = replace_entities(text)
print(processed_text)

герой вернулся в фильм Полицейский из локация


In [ ]:
df["description"] = df["description"].parallel_apply(lambda x: replace_entities(x))
df["plot"] = df["plot"].parallel_apply(lambda x: replace_entities(x))

In [ ]:
def remove_film_name_from_text(text, film_title):

    film_title_clean = re.sub(r"\(.*?\)", "", film_title).strip()
    text = text.replace(film_title, "фильм")
    text = text.replace(film_title_clean, "фильм")
    return " ".join(text.split())


remove_film_name_from_text(
    "Действие фильма «Дивергент» происходит в мире,", "Дивергент (2014)"
)

46 42


'Действие фильма «фильм» происходит в мире,'

In [ ]:
df["description"] = df.apply(
    lambda x: remove_film_name_from_text(x["description"], x["movie"]), axis=1
)
df["plot"] = df.apply(lambda x: remove_film_name_from_text(x["plot"], x["movie"]), axis=1)

1016 1012
190 190
1019 1023
195 195
939 939
372 372
281 281
235 235
214 214
465 465
520 520
640 640
316 316
1276 1276
365 365
573 573
79 79
479 471
522 522
520 520
555 555
579 579
527 527
541 541
343 343
168 168
476 476
439 439
634 634
461 461
319 319
575 575
323 323
681 681
412 412
419 419
586 586
304 304
371 371
131 131
279 279
140 140
228 228
859 859
557 557
575 575
323 323
550 550
422 422
498 498
491 491
338 338
395 395
567 567
235 235
662 662
403 403
419 419
290 290
324 324
407 407
318 318
396 396
389 389
302 302
386 386
372 372
303 303
635 635
242 242
330 330
312 312
462 462
597 597
323 323
526 526
1137 1137
529 529
530 530
403 403
287 287
406 406
361 361
550 550
179 179
393 393
283 274
228 228
759 759
592 592
564 564
374 374
341 341
442 442
396 396
186 186
490 490
181 181
460 460
678 678
290 290
183 183
808 808
321 321
308 308
123 123
1003 1003
521 521
167 167
865 865
259 259
441 441
258 258
476 476
885 885
240 240
263 263
354 354
383 383
457 457
717 717
468 468
574 574
404 404


In [ ]:
df

,movie,description,plot,target
0,Дивергент (2014),"Действие фильма «фильм» происходит в мире, пер...","Действие фильма «фильм» происходит в мире, пер...","[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]"
1,Кунг-фу Панда 4 (2024),Однажды ночью на вершине горы возле каменоломн...,Однажды ночью на вершине горы возле каменоломн...,"[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1]"
2,2046 (2004),герой возвращается в локация после нескольких ...,герой возвращается в локация после нескольких ...,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]"
3,Полицейский из Беверли-Хиллз: Аксель Фоули (2024),"герой вернулся в локация после того, как жизнь...","герой вернулся в локация после того, как жизнь...","[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"Знакомьтесь, Джо Блэк (1998)","История об герой, который решает взять отпуск ...","История об герой, который решает взять отпуск ...","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]"
...,...,...,...,...
566,Апокалипсис (2006),В 1517 году на полуострове локация племя герой...,1517 год. Полуостров локация. Группа охотников...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]"
567,Лёд 3 (2024),"герой, ставшая фигуристкой, стремится выиграть...",Фильм начинается с истории взросления дочери г...,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
568,Дастур (2023),"Новоиспеченная невеста, которую выдали замуж п...","Новоиспеченная невеста, которую выдали замуж п...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]"
569,Не говори никому (2024),Пара вместе с дочерью получают приглашение от ...,Пара вместе с дочерью получают приглашение от ...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"


In [ ]:
df = df.drop("movie", axis=1)

In [ ]:
df["description"].apply(len).max()

1707

In [ ]:
df["plot"].apply(len).max()

14681

In [ ]:
max_length = 2048
num_labels = len(mlb.classes_)
label2id = dict(zip(mlb.classes_, range(num_labels)))
id2label = dict(zip(range(num_labels), mlb.classes_))
model = "cointegrated/rubert-tiny2"
problem_type = "multi_label_classification"
batch_size = 16
pin_memory = False
drop_last = False
num_workers = 4
shuffle = True
val_size = int(len(df) * 0.1)
num_epochs = 30


os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)

model = BertForSequenceClassification.from_pretrained(
    model,
    num_labels=num_labels,
    problem_type=problem_type,
    label2id=label2id,
    id2label=id2label,
)

/home/seara/Desktop/Github/mtc-film-classification/.venv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.map(
    lambda x: tokenizer(
        x["description"], x["plot"], truncation=True, max_length=max_length
    ),
    batched=True,
)
dataset = dataset.map(
    lambda x: {"label": [float(y) for y in x["target"]]},
    batched=False,
    remove_columns=["description", "plot", "target"],
)

Map:   0%|          | 0/571 [00:00<?, ? examples/s]

Map:   0%|          | 0/571 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)


train_val = dataset.train_test_split(test_size=val_size)  # 10% of full dataset

train_dataloader = DataLoader(
    train_val["train"],
    batch_size=batch_size,
    shuffle=shuffle,
    num_workers=num_workers,
    collate_fn=data_collator,
    pin_memory=pin_memory,
    drop_last=drop_last,
)

val_dataloader = DataLoader(
    train_val["test"],
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=data_collator,
    pin_memory=pin_memory,
    drop_last=drop_last,
)

In [ ]:
THRESHOLD = 0.5


def get_report_multilabel(y_true, y_pred, target_names, output_dict):
    return classification_report(
        y_true,
        y_pred >= THRESHOLD,
        target_names=target_names,
        output_dict=output_dict,
        zero_division=0,
    )

In [ ]:
def predict(model, dataloader):
    with torch.inference_mode():
        y_true = []
        y_pred = []
        val_loss = 0

        for batch in tqdm(dataloader):
            batch = batch.to(model.device)
            output = model(**batch)
            loss = output.loss

            val_loss += loss.item() * batch["input_ids"].size(0)
            y_true.append(batch.labels.cpu())
            y_pred.append(output.logits.cpu())

        val_loss = val_loss / len(dataloader.dataset)

    return (
        torch.cat(y_true).numpy(),
        torch.sigmoid(torch.cat(y_pred)).numpy(),
        val_loss,
    )


def train_epoch(model, train_dataloader, optimizer):
    y_true = []
    y_pred = []
    train_loss = 0

    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        batch = batch.to(model.device)
        output = model(**batch)
        loss = output.loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * batch["input_ids"].size(0)
        y_true.append(batch.labels.detach().cpu())
        y_pred.append(output.logits.detach().cpu())

    train_loss = train_loss / len(train_dataloader.dataset)
    return (
        torch.cat(y_true).numpy(),
        torch.sigmoid(torch.cat(y_pred)).numpy(),
        train_loss,
    )


def eval(model, val_dataloader, labels):
    model.eval()
    val_y_true, val_y_pred, val_loss = predict(model, val_dataloader)
    report_dict = get_report_multilabel(val_y_true, val_y_pred, labels, True)
    df = pd.DataFrame(report_dict)
    df = df.round(2)
    return df

In [ ]:
def train(model, train_dataloader, optimizer, epochs, val_dataloader, labels):
    tq = tqdm(range(epochs))

    for epoch in tq:

        model.train()
        train_y_true, train_y_pred, train_loss = train_epoch(
            model, train_dataloader, optimizer
        )

        model.eval()
        val_y_true, val_y_pred, val_loss = predict(model, val_dataloader)

        tq.set_description(f"train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}")

    df = eval(model, val_dataloader, labels)
    print(df)

    return val_y_true, val_y_pred

In [ ]:
del nlp

torch.cuda.empty_cache()
model.cuda()

optimizer = Adam(model.parameters(), lr=0.00001)

val_y_true, val_y_pred = train(
    model=model,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    epochs=num_epochs,
    val_dataloader=val_dataloader,
    labels=id2label.values(),
)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

               0    1      2    3     4     5    6    7    8     9    10   11  \
precision   1.00  0.0   0.39  0.0   0.0   0.0  0.0  0.0  0.0   0.0   0.0  0.0   
recall      0.05  0.0   0.76  0.0   0.0   0.0  0.0  0.0  0.0   0.0   0.0  0.0   
f1-score    0.09  0.0   0.52  0.0   0.0   0.0  0.0  0.0  0.0   0.0   0.0  0.0   
support    22.00  5.0  21.00  7.0  22.0  12.0  6.0  6.0  3.0  16.0  14.0  6.0   

             12   13  micro avg  macro avg  weighted avg  samples avg  
precision   0.0  0.0       0.40       0.10          0.19         0.30  
recall      0.0  0.0       0.11       0.06          0.11         0.12  
f1-score    0.0  0.0       0.17       0.04          0.08         0.16  
support    11.0  9.0     160.00     160.00        160.00       160.00  


In [ ]:
submission = pd.read_csv("data/public_test.csv")
submission

,Фильм,Описание,Сюжет
0,Убойные каникулы (2010),Двое простых деревенских парней Дейл и Такер е...,Двое простых деревенских парней Дейл и Такер е...
1,"Три билборда на границе Эббинга, Миссури (2017)",После того как убийцы её дочери не были найден...,"В вымышленном городе Эббинг, штат Миссури, Мил..."
2,Глубоководный горизонт (2016),История о катастрофе на нефтяной платформе «Гл...,"20 апреля 2010 года Deepwater Horizon, нефтяна..."
3,Главный герой (2021),Парень живет в идеальном для себя мире — в луч...,Парень — неигровой персонаж (NPC) в «Фри Сити»...
4,Мистер и миссис Смит (2005),"Джон и Джейн, уставшие от однообразия брака, с...",Инженер-строитель Джон (Брэд Питт) и специалис...
...,...,...,...
117,Белая птица: Новое чудо (2023),Джулиана Олбанса исключили из школы за жестоко...,После событий «Чуда» Джулиан навсегда покинул ...
118,Манюня: Приключения в Москве (2024),"Девочки вместе с Ба едут на поезде в Москву, ч...","Девочки вместе с Ба едут на поезде в Москву, ч..."
119,Мадагаскар (2005),В зоопарке Центрального парка зебра Марти праз...,В зоопарке Центрального парка зебра Марти праз...
120,Взаперти (2020),"Диана Шерман, гиперзаботливая мать-одиночка, в...",Женщина по имени Диана Шерман преждевременно р...


In [ ]:
nlp = spacy.load("ru_core_news_lg")

submission = submission.rename(
    {"Фильм": "movie", "Сюжет": "plot", "Жанры": "genres", "Описание": "description"},
    axis=1,
)
submission["plot"] = submission["plot"].fillna(submission["description"])


submission["description"] = submission["description"].parallel_apply(
    lambda x: replace_entities(x)
)
submission["plot"] = submission["plot"].parallel_apply(lambda x: replace_entities(x))

submission["description"] = submission.apply(
    lambda x: remove_film_name_from_text(x["description"], x["movie"]), axis=1
)
submission["plot"] = submission.apply(
    lambda x: remove_film_name_from_text(x["plot"], x["movie"]), axis=1
)

submission = submission.drop("movie", axis=1)

/home/seara/Desktop/Github/mtc-film-classification/.venv/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'ru_core_news_lg' (3.7.0) was trained with spaCy v3.7.0 and may not be 100% compatible with the current version (3.8.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


287 287
641 641
298 281
645 645
406 406
278 278
368 368
484 484
264 264
704 704
618 618
501 501
219 219
607 607
296 296
520 520
260 260
300 300
151 151
529 529
581 581
354 354
299 299
350 350
619 619
383 383
1144 1144
642 642
474 474
306 306
447 447
254 254
216 216
379 377
352 352
354 354
220 220
721 721
508 508
356 356
316 312
538 538
458 458
480 480
436 436
1117 1117
630 630
416 416
186 186
595 595
807 807
363 363
572 572
451 451
771 771
381 381
160 160
158 158
437 437
351 351
213 213
449 449
520 520
404 404
393 393
693 693
592 592
522 522
443 443
408 408
437 437
342 342
672 672
535 535
410 410
499 499
452 452
596 592
322 322
877 875
578 578
423 423
270 270
760 757
250 250
398 398
548 548
831 831
479 479
350 350
150 150
362 362
258 258
347 347
616 616
1281 1281
369 369
254 254
272 272
335 335
466 466
540 540
367 367
526 526
463 463
128 128
261 261
468 467
345 345
148 148
426 426
553 553
291 288
479 479
387 387
256 257
479 479
293 293
972 972
1035 1035
574 574
195 195
2553 2553
4488 4

In [ ]:
dataset = Dataset.from_pandas(submission)
dataset = dataset.map(
    lambda x: tokenizer(
        x["description"], x["plot"], truncation=True, max_length=max_length
    ),
    batched=True,
    remove_columns=["description", "plot"],
)

test_dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=data_collator,
    pin_memory=pin_memory,
    drop_last=drop_last,
)


def test(model, dataloader):
    with torch.inference_mode():
        y_pred = []

        for batch in tqdm(dataloader):
            batch = batch.to(model.device)
            output = model(**batch)

            y_pred.append(output.logits.cpu())

    return torch.sigmoid(torch.cat(y_pred)).numpy()

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

In [ ]:
prediction = test(model, test_dataloader)

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
binary_predictions = (prediction >= 0.05).astype(int)

# Get the predicted labels for each sample
predicted_labels = []
for pred in binary_predictions:
    labels = [id2label[idx] for idx, value in enumerate(pred) if value == 1]
    predicted_labels.append(labels)

NameError: name 'prediction' is not defined

In [ ]:
submission = pd.read_csv("data/sample_submission_seed_0.csv")
submission["Жанры"] = predicted_labels
submission["Жанры"] = submission["Жанры"].apply(lambda x: ", ".join(x))

In [ ]:
submission.to_csv("rubert_tiny_2.csv", index=False)